In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob

In [2]:
DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

In [3]:
train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

In [4]:
from keras.preprocessing.text import Tokenizer

MAX_WORD_NUMS = 10000

word_tokenizer = Tokenizer(MAX_WORD_NUMS)
word_tokenizer.fit_on_texts(question_data["words"])

Using TensorFlow backend.


In [5]:
word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_WORD_NUMS]].values
    ),
    axis=0
)
word_embedding_data.shape

(10001, 300)

In [6]:
from keras.preprocessing.sequence import pad_sequences

WORD_SEQ_LEN = 30

def gen_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre")

word1, word2 = gen_data(train_data)
test_word1, test_word2 = gen_data(test_data)

word1.shape, word2.shape, test_word1.shape, test_word2.shape

((254386, 30), (254386, 30), (172956, 30), (172956, 30))

In [ ]:
label = train_data["label"].values

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.layers import Input, Embedding, Conv1D, GlobalAveragePooling1D, Lambda, Dropout, BatchNormalization, Dense, K
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import concatenate

CONV_LEN_1 = 128
CONV_LEN_2 = 128
CONV_LEN_3 = 128
CONV_LEN_4 = 128
CONV_LEN_5 = 128
CONV_LEN_6 = 128
CONV_LEN = CONV_LEN_1 + CONV_LEN_2 + CONV_LEN_3 + CONV_LEN_4 + CONV_LEN_5 + CONV_LEN_6
DROP_RATE = 0.7
DENSE_SIZE = 256
BATCH_SIZE = 2048
NUM_EPOCHES = 100

pred_collect = []

for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10).split(X=word1, y=label)):
    train_word1, train_word2, train_y = word1[train_index, :], word2[train_index, :], label[train_index]
    dev_word1, dev_word2, dev_y = word1[dev_index, :], word2[dev_index, :], label[dev_index]
    
    word_input1 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
    word_input2 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
    
    embedding_layer = Embedding(
        input_dim=word_embedding_data.shape[0],
        output_dim=word_embedding_data.shape[1],
        weights=[word_embedding_data],
        input_length=WORD_SEQ_LEN,
        trainable=False
    )
    
    word_vector1 = embedding_layer(word_input1)
    word_vector2 = embedding_layer(word_input2)
    
    conv1 = Conv1D(filters=128, kernel_size=1, padding="same", activation="relu")
    conv1a = conv1(word_vector1)
    conv1a = GlobalAveragePooling1D()(conv1a)
    conv1b = conv1(word_vector2)
    conv1b = GlobalAveragePooling1D()(conv1b)
    
    conv2 = Conv1D(filters=128, kernel_size=2, padding="same", activation="relu")
    conv2a = conv2(word_vector1)
    conv2a = GlobalAveragePooling1D()(conv2a)
    conv2b = conv2(word_vector2)
    conv2b = GlobalAveragePooling1D()(conv2b)
    
    conv3 = Conv1D(filters=128, kernel_size=3, padding="same", activation="relu")
    conv3a = conv3(word_vector1)
    conv3a = GlobalAveragePooling1D()(conv3a)
    conv3b = conv3(word_vector2)
    conv3b = GlobalAveragePooling1D()(conv3b)
    
    conv4 = Conv1D(filters=128, kernel_size=4, padding="same", activation="relu")
    conv4a = conv4(word_vector1)
    conv4a = GlobalAveragePooling1D()(conv4a)
    conv4b = conv4(word_vector2)
    conv4b = GlobalAveragePooling1D()(conv4b)
    
    conv5 = Conv1D(filters=128, kernel_size=5, padding="same", activation="relu")
    conv5a = conv5(word_vector1)
    conv5a = GlobalAveragePooling1D()(conv5a)
    conv5b = conv5(word_vector2)
    conv5b = GlobalAveragePooling1D()(conv5b)
    
    conv6 = Conv1D(filters=128, kernel_size=6, padding="same", activation="relu")
    conv6a = conv6(word_vector1)
    conv6a = GlobalAveragePooling1D()(conv6a)
    conv6b = conv6(word_vector2)
    conv6b = GlobalAveragePooling1D()(conv6b)
    
    merge_a = concatenate([conv1a, conv2a, conv3a, conv4a, conv5a, conv6a])
    merge_b = concatenate([conv1b, conv2b, conv3b, conv4b, conv5b, conv6b])
    
    diff = Lambda(lambda x: x[0] - x[1], output_shape=(CONV_LEN,))([merge_a, merge_b])
    mult = Lambda(lambda x: x[0] * x[1], output_shape=(CONV_LEN,))([merge_a, merge_b])
    
    merge = concatenate([diff, mult])
    
    x = Dropout(DROP_RATE)(merge)
    x = BatchNormalization()(x)
    x = Dense(DENSE_SIZE, activation="relu")(x)
    
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)
    pred = Dense(1, activation="sigmoid")(x)
    
    model = Model(
        inputs = [word_input1, word_input2],
        outputs = pred
    )
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["acc"]
    )
    
    early_stop = EarlyStopping("val_loss", patience=10)
    check_point = ModelCheckpoint(
        "./log/cnn_%02d.{epoch:02d}_{val_loss:.3f}.hdf5" % (i + 1),
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True
    )
    
    model_res = model.fit(
        x=[train_word1, train_word2],
        y=train_y,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHES,
        validation_data=([dev_word1, dev_word2], dev_y),
        shuffle=True,
        callbacks=[early_stop, check_point]
    )
    
    print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
    model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))

    test_pred = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)
    pred_collect.append(pd.DataFrame(test_pred, columns=["y_pre"]))

pd.DataFrame(pd.concat(pred_collect, axis=1).mean(axis=1), columns=["y_pre"]).to_csv("./result/pred.csv", index=False)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 228946 samples, validate on 25440 samples
Epoch 1/100
228946/228946 [==============================] - 32s 138us/step - loss: 0.5858 - acc: 0.7159 - val_loss: 0.4409 - val_acc: 0.7996
Epoch 2/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.4444 - acc: 0.7962 - val_loss: 0.4013 - val_acc: 0.8162
Epoch 3/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.3996 - acc: 0.8195 - val_loss: 0.3597 - val_acc: 0.8376
Epoch 4/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.3715 - acc: 0.8352 - val_loss: 0.3308 - val_acc: 0.8513
Epoch 5/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.3529 - acc: 0.8440 - val_loss: 0.3159 - val_acc: 0.8599
Epoch 6/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.3392 - acc: 0.8522 - val_loss: 0.3027 - val_acc: 0.8671
Epoch 7/100
22

Epoch 56/100
228946/228946 [==============================] - 32s 141us/step - loss: 0.1937 - acc: 0.9206 - val_loss: 0.2290 - val_acc: 0.8996
Epoch 57/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.1924 - acc: 0.9217 - val_loss: 0.2257 - val_acc: 0.9014
Epoch 58/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.1910 - acc: 0.9221 - val_loss: 0.2236 - val_acc: 0.9026
Epoch 59/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.1912 - acc: 0.9221 - val_loss: 0.2296 - val_acc: 0.8989
Epoch 60/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.1883 - acc: 0.9237 - val_loss: 0.2198 - val_acc: 0.9062
Epoch 61/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.1886 - acc: 0.9233 - val_loss: 0.2294 - val_acc: 0.8994
Epoch 62/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.1880 - acc: 0.9230 - val_loss: 0.2258 - val_acc: 0.9006

Epoch 33/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.2238 - acc: 0.9073 - val_loss: 0.2429 - val_acc: 0.8918
Epoch 34/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.2218 - acc: 0.9082 - val_loss: 0.2314 - val_acc: 0.8988
Epoch 35/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.2186 - acc: 0.9095 - val_loss: 0.2345 - val_acc: 0.8971
Epoch 36/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.2170 - acc: 0.9113 - val_loss: 0.2297 - val_acc: 0.9002
Epoch 37/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.2160 - acc: 0.9105 - val_loss: 0.2339 - val_acc: 0.8963
Epoch 38/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.2143 - acc: 0.9114 - val_loss: 0.2316 - val_acc: 0.8983
Epoch 39/100
228946/228946 [==============================] - 29s 126us/step - loss: 0.2143 - acc: 0.9119 - val_loss: 0.2338 - val_acc: 0.8975

Epoch 31/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2287 - acc: 0.9055 - val_loss: 0.2328 - val_acc: 0.8989
Epoch 32/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2256 - acc: 0.9064 - val_loss: 0.2459 - val_acc: 0.8890
Epoch 33/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2231 - acc: 0.9073 - val_loss: 0.2349 - val_acc: 0.8973
Epoch 34/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2208 - acc: 0.9081 - val_loss: 0.2307 - val_acc: 0.9004
Epoch 35/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2195 - acc: 0.9093 - val_loss: 0.2343 - val_acc: 0.8969
Epoch 36/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2183 - acc: 0.9094 - val_loss: 0.2351 - val_acc: 0.8963
Epoch 37/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2181 - acc: 0.9092 - val_loss: 0.2297 - val_acc: 0.8999

Epoch 35/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2226 - acc: 0.9078 - val_loss: 0.2277 - val_acc: 0.9000
Epoch 36/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2198 - acc: 0.9087 - val_loss: 0.2290 - val_acc: 0.8986
Epoch 37/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2192 - acc: 0.9093 - val_loss: 0.2280 - val_acc: 0.9008
Epoch 38/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2171 - acc: 0.9104 - val_loss: 0.2308 - val_acc: 0.8992
Epoch 39/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2168 - acc: 0.9102 - val_loss: 0.2281 - val_acc: 0.9011
Epoch 40/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2143 - acc: 0.9110 - val_loss: 0.2285 - val_acc: 0.8989
Epoch 41/100
228947/228947 [==============================] - 29s 127us/step - loss: 0.2129 - acc: 0.9121 - val_loss: 0.2242 - val_acc: 0.9017

Epoch 8/100
228948/228948 [==============================] - 29s 129us/step - loss: 0.3144 - acc: 0.8636 - val_loss: 0.2884 - val_acc: 0.8735
Epoch 9/100
228948/228948 [==============================] - 30s 129us/step - loss: 0.3070 - acc: 0.8673 - val_loss: 0.2827 - val_acc: 0.8741
Epoch 10/100
228948/228948 [==============================] - 30s 130us/step - loss: 0.2990 - acc: 0.8709 - val_loss: 0.2766 - val_acc: 0.8778
Epoch 11/100
228948/228948 [==============================] - 30s 130us/step - loss: 0.2932 - acc: 0.8737 - val_loss: 0.2765 - val_acc: 0.8750
Epoch 12/100
228948/228948 [==============================] - 30s 129us/step - loss: 0.2861 - acc: 0.8773 - val_loss: 0.2691 - val_acc: 0.8796
Epoch 13/100
228948/228948 [==============================] - 29s 128us/step - loss: 0.2811 - acc: 0.8799 - val_loss: 0.2643 - val_acc: 0.8825
Epoch 14/100
228948/228948 [==============================] - 30s 129us/step - loss: 0.2758 - acc: 0.8821 - val_loss: 0.2599 - val_acc: 0.8847
E